In [18]:
import os
from dotenv import load_dotenv

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI

In [19]:
load_dotenv()

True

In [20]:
current_path = os.getcwd()
user_home_path = os.path.expanduser("~")

In [21]:
model = ChatOpenAI(model="gpt-4o", temperature=0.0)
command = f"{user_home_path}/.virtualenvs/generative_ai/bin/python3"
math_sever = f"{current_path}/mcp_adapters_math_server.py"

In [22]:
# Create server parameters for stdio connection
server_params = StdioServerParameters(
    command=command,
    # Make sure to update to the full absolute path to your math_server.py file
    args=[math_sever],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent(model, tools)
        agent_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})

In [24]:
async with MultiServerMCPClient(
    {
        "math": {
            "command": command,
            # Make sure to update to the full absolute path to your math_server.py file
            "args": [math_sever],
            "transport": "stdio",
        },
        "weather": {
            # make sure you start your weather server on port 8000
            "url": "http://localhost:8000/sse",
            "transport": "sse",
        }
    }
) as client:
    agent = create_react_agent(model, client.get_tools())
    math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
    weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})